To make a kernel with a siphon virtualenv from the terminal:

```
conda create -n siphon siphon
conda activate siphon
conda install ipykernel
python -m ipykernel install --user --name=siphon
```

Then select the `siphon` from the pulldown/window in Jupyter.

In [ ]:
from siphon.simplewebservice.wyoming import WyomingUpperAir
from datetime import datetime
import pandas

start_time = datetime(2023, 3, 1, 0)
end_time = datetime(2023, 6, 30, 0)

station = "43371" # Trivandrum, Thiruvananthapuram, 'VOTX'?

df = WyomingUpperAir.request_data(start_time, station) # returns Pandas dataframe

In [ ]:
df

In [ ]:
# for column in df:
#     print( df[column].name )
#     print( df[column].values )

# df[column].shape[0]
df[df.columns[0:8]]

# skip column 8

# scalars


df.columns[9]
type(df[df.columns[9]]) is pandas.core.series.Series

df[df.columns[10]][0].strftime('%Y-%m-%d %H:%M:%S') # convert Datestamp to string
type(df[df.columns[10]][0]) is pandas._libs.tslibs.timestamps.Timestamp
    

In [ ]:
# # bad way... & broken

# #from siphon.cdmr import Dataset
# from siphon.ncss import Dataset
# from io import StringIO

# # Convert the DataFrame to CSV format in memory
# csv_buffer = StringIO()
# df.to_csv(csv_buffer, sep=',', index=False)
# csv_buffer.getvalue().splitlines()[0].encode()
# csv_buffer.getvalue().splitlines()[1:]

# # Open a Dataset object and write the CSV data to it
# nc = Dataset('sounding.nc', 'w', format='NETCDF4')
# nc.createDimension('index', None)
# nc.createVariable('header', 'S1', ('header',))
# # nc.variables['header'][0] = csv_buffer.getvalue().splitlines()[0].encode()
# # nc.variables['header'][0] = 
# nc.createVariable('data', 'f8', ('index', 'header'))

# for i, column in enumerate(df):
#     # print( df[column].name )
#     nc.variables['header'][i] = df[column].name
#     # print( df[column].values )
    
# for i, line in enumerate(csv_buffer.getvalue().splitlines()[1:]):
#     #nc.variables['data'][i, :] = line.split(',') # can't autoconvert string to float
#     line.split(',')
# nc.close()


In [ ]:
from netCDF4 import Dataset

"convert Timestamps into string, otherwise just return input."
def Timestamp2String( t ):
    return ( t.strftime('%Y-%m-%d %H:%M:%S') if type(t) is pandas._libs.tslibs.timestamps.Timestamp else t )

"write a sounding dataframe as a NetCDF4 file."
def sounding2nc( df, filename ):

    # Open a new NetCDF file for writing
    nc_file = Dataset(filename, mode='w')

    # Set the attributes from df cols 9...
    for column in df.columns[9:]:
        #  print( "set attribute "+df[column].name+"=", Timestamp2String(df[column][0]) )
        nc_file.setncattr( df[column].name, Timestamp2String(df[column][0]) )

    # or set all at once from a dict: setncatts(self,attdict)

    # Define dimensions for pressure, temperature, etc. variables
    time_dim = nc_file.createDimension('time', 1)
    level_dim = nc_file.createDimension('level', df[df.columns[0]].shape[0])

    units_list = [ 'hPa', 'm', 'degrees C', 'degrees C', 'degrees', 'm/s', 'm/s', 'm/s' ]

    # Create variables for time, level, latitude, longitude, pressure, and temperature
    for i, column in enumerate( df.columns[0:7] ):
        # create the netcdf variable
        var = nc_file.createVariable(df[column].name, 'f4', ('time', 'level'))
        # write the data to the NetCDF file
        var[:] = df[column][:]
        # supply units attributes
        var.units = units_list[i]

    # Close the NetCDF file
    return nc_file.close()

In [155]:
# get Trivandrum soundings from 2019, write to netcdf

start_time = datetime(2019, 3, 1, 0)
end_time = datetime(2019, 6, 30, 0)
station = "43371" # Trivandrum, Thiruvananthapuram, 'VOTX'?

for dt in pandas.date_range(start_time, end_time, freq='12H'):
    try:
        df = WyomingUpperAir.request_data(dt, station) # returns Pandas dataframe
        sounding2nc( df, "../data/uwyo/trivandrum/Trivandrum"+df.time[0].strftime('%Y%m%d_%H%M')+".nc" )
    except:
        continue
    else:
        continue
        



In [ ]:
station = "43063"  # Pune
for dt in pandas.date_range(start_time, end_time, freq='12H'):
    try:
        df = WyomingUpperAir.request_data(dt, station) # returns Pandas dataframe
        sounding2nc( df, "../data/uwyo/pune/Pune"+df.time[0].strftime('%Y%m%d_%H%M')+".nc" )
    except:
        continue

In [ ]:
df = WyomingUpperAir.request_data(dt, station)